In [1]:
import os 
import sys
from tqdm import tqdm
import importlib
import numpy as np
import pickle
import tensorflow as tf
import matplotlib.pyplot as plt

module_path = "/home/lun/project-basileus/multitype-sequence-generation-by-tlstm-gan/"
data_path = "/home/lun/project-basileus/multitype-sequence-generation-by-tlstm-gan/data/long_seqs_v10/"

if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
tf.__version__

'2.1.0'

## Load Data

In [3]:
pos_data_path = os.path.join(data_path, 'positive_long_sequences.pickle')
neg_data_path = os.path.join(data_path, 'negative_long_sequences.pickle')

def load_sequence_from_pickle_to_numpy(pickle_file_path):
    """
        A list of sequence in format of (event_type, time_delta)
    :param pickle_file_path: e.g. /.../project-basileus/seq-gan/data/fixed_length/valid_sequences.pickle
    :return: (event_type_seqs, time_delta)
    """
    with open(pickle_file_path, 'rb') as f:
        raw_seqs = pickle.load(f)

    if not raw_seqs or not raw_seqs[0]:
        return np.array([]), np.array([])

    N = len(raw_seqs)
    T = len(raw_seqs[0])
    
    seqs = np.array(raw_seqs)
#     print(seqs.shape)
    
    et_seqs = seqs[:, :, 0].astype(np.float64).reshape((N, T, 1))
    ts_seqs = seqs[:, :, 1].astype(np.float64).reshape((N, T, 1))
    return et_seqs, ts_seqs
    
pos_event_type_seqs, pos_timestamp_seqs = load_sequence_from_pickle_to_numpy(pos_data_path)
neg_event_type_seqs, neg_timestamp_seqs = load_sequence_from_pickle_to_numpy(neg_data_path)

In [4]:
def create_dataset(features: np.array, labels: np.array, batch_size=2, epochs=10, buffer_size=10000):
    """
    Create dataset from numpy arrays
    """
    dataset = tf.data.Dataset.from_tensor_slices((features, labels))
    dataset = dataset.shuffle(buffer_size)
    dataset = dataset.repeat(epochs)
    dataset = dataset.batch(batch_size, drop_remainder=True)
    return dataset

In [5]:
BATCH_SIZE = 16
T = 20 + 1
VOCAB = ['END/PADDING', 'INIT', 'start', 'view', 'click', 'install']
EVENT_VOCAB_DIM = len(VOCAB)
EMB_DIM = 8
HIDDEN_DIM = 32

END_TOKEN = 0
MAX_TIME = 1024

## Create original SeqGan

In [6]:
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model
from tensorflow.keras import Sequential
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.layers import Input, LSTM, Embedding, Reshape, Dense, Dropout, Conv1D, Conv2D, MaxPooling1D, Activation, Multiply, Add, Lambda
from tensorflow.keras import regularizers

# from sgtlstm.TimeLSTM import TimeLSTM0, TimeLSTM1, TimeLSTM2, TimeLSTM3

tf.keras.backend.set_floatx('float64')


def build_G(batch_size, event_vocab_dim, emb_dim, hidden_dim=11):
    # normal LSTM
    i_et = Input(batch_shape=(batch_size, None, 1), name='event_type')  # input of discrete feature event type
    mask_layer = tf.keras.layers.Masking(mask_value=0., input_shape=(None, 1))
    masked_et = mask_layer(i_et)
    
    embed0 = Embedding(input_dim=event_vocab_dim, output_dim=emb_dim, mask_zero=True)(masked_et)
    # turn [batch_size, None, 1, emb_dim] -> [batch_size, None, emb_dim]
    embed0 = tf.squeeze(embed0, axis=2)

    hm = LSTM(hidden_dim, 
              name='lstm',
              stateful=True,
              return_sequences=False,
              kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.1),
              recurrent_initializer=tf.keras.initializers.RandomNormal(stddev=0.1),
              bias_initializer=tf.keras.initializers.RandomNormal(stddev=0.1))(embed0)

    logits = Dense(event_vocab_dim,
                   activation='linear',
                   name='dense_1',
                   kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.1),
                   bias_initializer=tf.keras.initializers.RandomNormal(stddev=0.1))(hm)
    
    generator = Model(
        inputs=i_et,
        outputs=logits)
        
    return generator

In [7]:
# Modified from: https://gist.github.com/iskandr/a874e4cf358697037d14a17020304535
def highway_layers(value, n_layers, activation="tanh", gate_bias=-3):
    dim = K.int_shape(value)[-1]
    gate_bias_initializer = tf.keras.initializers.Constant(gate_bias)
    for i in range(n_layers):     
        gate = Dense(units=dim, bias_initializer=gate_bias_initializer, activation='sigmoid')(value)
        negated_gate = Lambda(
            lambda x: 1.0 - x,
            output_shape=(dim,))(gate)
        transformed = Dense(units=dim, activation=activation)(value)
        transformed_gated = Multiply()([gate, transformed])
        identity_gated = Multiply()([negated_gate, value])
        value = Add()([transformed_gated, identity_gated])
    return value

In [8]:
def build_D(T, event_vocab_dim, emb_dim, num_filters=100, kernel_size=4, dropout_rate=0.25):
    # Highway network
    i_et = Input(shape=(T, 1), name='event_type')  # input of discrete feature event type

    mask_layer = tf.keras.layers.Masking(mask_value=0., input_shape=(T, 1))
    masked_et = mask_layer(i_et)
    
    embed0 = Embedding(input_dim=event_vocab_dim, output_dim=emb_dim, mask_zero=True)(masked_et)
    embed0 = tf.squeeze(embed0, axis=2)

    conv1 = Conv1D(
        filters=num_filters,
        kernel_size=kernel_size,
        strides=1,
        padding="valid",
        activation='relu',
        use_bias=True,
        kernel_initializer=tf.keras.initializers.TruncatedNormal(mean=0.0, stddev=0.1, seed=None),
        bias_initializer=tf.keras.initializers.Constant(value=0.1))(embed0)
    
    pooled = MaxPooling1D(pool_size=T-kernel_size+1, strides=1)(conv1)
    
    highwayed = highway_layers(pooled, n_layers=1, activation="relu", gate_bias=0)

    dropped = Dropout(rate=dropout_rate)(highwayed)
    
    prob = Dense(1, 
                 activation='sigmoid',
                 name='final',
                 kernel_initializer=tf.keras.initializers.TruncatedNormal(mean=0.0, stddev=0.1, seed=None),
                 bias_initializer=tf.keras.initializers.Constant(value=0.1))(dropped)

    discriminator = Model(
        inputs=i_et,
        outputs=prob)

    return discriminator

## Test initial G and D

In [9]:
G = build_G(batch_size=BATCH_SIZE, event_vocab_dim=11, emb_dim=EMB_DIM, hidden_dim=HIDDEN_DIM)

In [10]:
seq_batch_1 = tf.cast(tf.random.uniform([BATCH_SIZE, T-1, 1], maxval=10, dtype=tf.int32)+1, tf.float32)
# seq_batch_1

In [11]:
G(seq_batch_1)


If you intended to run this layer in float64, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float32 by default, call `tf.keras.backend.set_floatx('float32')`. To change just this layer, pass dtype='float32' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



<tf.Tensor: shape=(16, 11), dtype=float64, numpy=
array([[ 0.1765201 ,  0.05585903,  0.20178451,  0.03028732,  0.12424717,
         0.03789527,  0.09946218,  0.12018467,  0.13959934, -0.00916542,
        -0.09173811],
       [ 0.1744123 ,  0.05578895,  0.20234678,  0.03416886,  0.12248743,
         0.03653074,  0.09624811,  0.12222482,  0.13825018, -0.0033136 ,
        -0.08937169],
       [ 0.1746303 ,  0.05604111,  0.20182739,  0.03341078,  0.12223501,
         0.03858539,  0.09676657,  0.12269244,  0.1404487 , -0.00445556,
        -0.09197856],
       [ 0.1765899 ,  0.05554279,  0.20235011,  0.03034806,  0.12482255,
         0.0381483 ,  0.09910955,  0.12005635,  0.13998159, -0.00913074,
        -0.09137338],
       [ 0.17619801,  0.05436476,  0.20174684,  0.03015842,  0.12355863,
         0.03799207,  0.09889742,  0.11980309,  0.13967171, -0.00716463,
        -0.09120128],
       [ 0.17682136,  0.05610518,  0.20122717,  0.02907545,  0.12396806,
         0.03878195,  0.10027924,  0.

In [12]:
D = build_D(T=20, event_vocab_dim=11, emb_dim=32, num_filters=100, kernel_size=4, dropout_rate=0.25)

In [13]:
D(seq_batch_1)


If you intended to run this layer in float64, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float32 by default, call `tf.keras.backend.set_floatx('float32')`. To change just this layer, pass dtype='float32' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



<tf.Tensor: shape=(16, 1, 1), dtype=float64, numpy=
array([[[0.5169389 ]],

       [[0.51490091]],

       [[0.51196267]],

       [[0.51233631]],

       [[0.51811637]],

       [[0.51772811]],

       [[0.51480239]],

       [[0.5152546 ]],

       [[0.51446446]],

       [[0.51678611]],

       [[0.51403316]],

       [[0.51117157]],

       [[0.51398358]],

       [[0.5146369 ]],

       [[0.51410222]],

       [[0.51170276]]])>

## Pre-Training of original SeqGAn

In [14]:
def pretrain_discriminator(event_type_batch, label_batch, discriminator, verbose=False, optimizer=Adam(lr=0.001)):
    # train the discriminator
    with tf.GradientTape() as tape:
        # train discriminator
        true_prob = discriminator(event_type_batch)

        # cross-entropy loss
        discriminator_loss = ce_loss = tf.reduce_mean(
            tf.keras.losses.binary_crossentropy(label_batch, true_prob, from_logits=False)
        )

        if verbose:
            print('total discriminator loss:{}'.format(discriminator_loss))

    grads = tape.gradient(discriminator_loss, discriminator.trainable_variables)
    optimizer.apply_gradients(zip(grads, discriminator.trainable_variables))

    return discriminator_loss


def pretrain_generator(event_type_batch, generator, verbose=False, optimizer=Adam(lr=0.001)):
    _, T, _ = event_type_batch.shape

    # train the generator
    with tf.GradientTape() as tape:
        generator.reset_states()
        ce_loss_list = []

        for i in range(0, T - 1):
            curr_state_et = event_type_batch[:, i:i + 1, :]
            target_et = event_type_batch[:, i + 1, :]

            token_logits = generator(curr_state_et)

            ce_losses = tf.keras.losses.sparse_categorical_crossentropy(target_et, token_logits, from_logits=True)
            ce_loss = tf.reduce_mean(ce_losses)
            ce_loss_list.append(ce_loss)

        generator_loss = tf.reduce_mean(ce_loss_list)

    if verbose:
        print('train loss:{}'.format(generator_loss))

    # apply gradient decent per batch
    grads = tape.gradient(generator_loss, generator.trainable_variables)
    optimizer.apply_gradients(zip(grads, generator.trainable_variables))

    return generator_loss

### pre-train G

In [15]:
N_total_G = pos_event_type_seqs.shape[0]

EPOCHS = 1
_TOTAL_STEPS = int(EPOCHS * N_total_G / BATCH_SIZE)

pretrain_G_dataset = create_dataset(pos_event_type_seqs,
                                  np.ones((N_total_G, 1)),
                                  batch_size=BATCH_SIZE,
                                  epochs=EPOCHS,
                                  buffer_size=N_total_G)

In [16]:
pretrain_gen_ce_loss_history = []
pretrain_gen_metrics_history = []

pretrained_generator = build_G(
    batch_size=BATCH_SIZE,
    event_vocab_dim = EVENT_VOCAB_DIM,
    emb_dim = EMB_DIM,
    hidden_dim= HIDDEN_DIM)

In [17]:
step = 0
OPTIMIZER = Adam(lr=1e-3)

for event_type_batch, _ in tqdm(pretrain_G_dataset.take(_TOTAL_STEPS)):
    step += 1
    print('Training Step:', step)
        
    gen_ce_loss =  pretrain_generator(event_type_batch, pretrained_generator, verbose=True, optimizer=OPTIMIZER)                    
    pretrain_gen_ce_loss_history.append(gen_ce_loss.numpy())

0it [00:00, ?it/s]

Training Step: 1
train loss:1.8376708021325405


1it [00:00,  2.09it/s]

Training Step: 2
train loss:1.8292898793685217


3it [00:00,  3.77it/s]

...

Training Step: 30
train loss:1.6942935450274756
Training Step: 31


31it [00:06,  5.02it/s]

train loss:1.6908118598947133
Training Step: 32
train loss:1.6792085539263781


31it [00:06,  4.67it/s]


KeyboardInterrupt: 

In [ ]:
model_save_dir = '/home/lun/project-basileus/multitype-sequence-generation-by-tlstm-gan/models/seqgan_original'
if not os.path.exists(model_save_dir + '/pretrained_gen_weights'):
    os.makedirs(model_save_dir + '/pretrained_gen_weights')

G_save_path = model_save_dir + '/pretrained_gen_weights/model.tf'
pretrained_generator.save_weights(G_save_path)

In [ ]:
reload_pretrained_gen = build_G(
    batch_size = BATCH_SIZE,
    event_vocab_dim = EVENT_VOCAB_DIM,
    emb_dim = EMB_DIM,
    hidden_dim= HIDDEN_DIM,
)

reload_pretrained_gen.build(input_shape=((BATCH_SIZE, T, 1)))
reload_pretrained_gen.load_weights(G_save_path)
reload_pretrained_gen.summary()

In [ ]:
def rollout_from_prefix(
        G, batch_size, prefix, L=T, end_token=0, init_token=1):
    # Begin from dummy init state (init_token=1, init_timestamp=0.0)
    all_state_et = prefix
    curr_state_et = prefix[:, -1:, :]
    l_prefix = prefix.shape[1]
    
    G.reset_states()

    for _ in range(L-l_prefix):  # sequence length
        token_logits = G(curr_state_et)
        
        sampled_et = tf.random.categorical(token_logits, num_samples=1, dtype=tf.int32)
        sampled_et = tf.reshape(sampled_et, [batch_size, 1, 1])

        # cast sampled_et into float
        sampled_et = tf.cast(sampled_et, dtype=tf.float64)

        # stop genererating once hit end_token
        cond_end_token = tf.equal(curr_state_et, end_token)
        curr_state_et = tf.where(cond_end_token, curr_state_et, sampled_et)
        all_state_et = tf.concat([all_state_et, curr_state_et], axis=1)

    return all_state_et



def generate_sequences(N_gen, generator, batch_size, T):
    """
        Generate sequences batch per batch
    :param N_gen: total number of seqs to be generated
    :param generator:
    :param batch_size:
    :param T:
    :return: a python list of shape [N_gen, T, 1]
    """
    N = 0
    all_type_seq = None
    init_prefix = tf.ones([batch_size, 1, 1], dtype=tf.float64)
    
    while N < N_gen:
        batch_state_et = rollout_from_prefix(generator, batch_size, init_prefix, T)
        batch_type_seq = batch_state_et.numpy()

        if all_type_seq is None:
            all_type_seq = batch_type_seq
        else:
            all_type_seq = np.concatenate([all_type_seq, batch_type_seq], axis=0)

        N += batch_size

    # convert to python list
    type_seq_list = all_type_seq[:N_gen].tolist()

    return type_seq_list

### pre-train D

In [ ]:
N_fake_D = N_real_D = N_total_G // 100

In [ ]:
# generate fake data by G to pre-train D
fake_pos_event_type_seqs = np.array(generate_sequences(N_fake_D, pretrained_generator, BATCH_SIZE, T))

In [ ]:
# generate fake data by G to pre-train D
real_pos_event_type_seqs = pos_event_type_seqs[0:N_real_D, :, :]

In [ ]:
N_total_D = N_fake_D + N_real_D

pretrain_D_et = np.concatenate([fake_pos_event_type_seqs, real_pos_event_type_seqs], axis=0)
pretrain_D_labels = np.concatenate([np.zeros((N_fake_D, 1)), np.ones((N_real_D, 1))], axis=0)

EPOCHS = 10
_TOTAL_STEPS = int(EPOCHS * N_total_D / BATCH_SIZE)

pretrain_D_dataset = create_dataset(pretrain_D_et,
                                    pretrain_D_labels,
                                    batch_size=BATCH_SIZE,
                                    epochs=EPOCHS,
                                    buffer_size=N_total_D) # shuffle the entire Dataset

In [ ]:
pretrain_disc_token_loss_history = []

pretrained_discriminator = build_D(T=T,
                                   event_vocab_dim=EVENT_VOCAB_DIM,
                                   emb_dim=EMB_DIM,
                                   num_filters=100,
                                   kernel_size=4,
                                   dropout_rate=0.25)

In [ ]:
step = 0
OPTIMIZER = SGD(lr=1e-3)
for event_type_batch, labels in tqdm(pretrain_D_dataset.take(_TOTAL_STEPS)):
    step += 1
    print('Training Step:', step)

    disc_token_loss = pretrain_discriminator(event_type_batch, labels, pretrained_discriminator, verbose=True, optimizer=OPTIMIZER)
    pretrain_disc_token_loss_history.append(disc_token_loss.numpy())

In [ ]:
model_save_dir = '/home/lun/project-basileus/multitype-sequence-generation-by-tlstm-gan/models/seqgan_original'
if not os.path.exists(model_save_dir + '/pretrained_disc_weights'):
    os.makedirs(model_save_dir + '/pretrained_disc_weights')

D_save_path = model_save_dir + '/pretrained_disc_weights/model.tf'
pretrained_discriminator.save_weights(D_save_path)

In [ ]:
reload_pretrained_disc = build_D(T=T,
                                   event_vocab_dim=EVENT_VOCAB_DIM,
                                   emb_dim=EMB_DIM,
                                   num_filters=100,
                                   kernel_size=4,
                                   dropout_rate=0.25)

reload_pretrained_disc.build(input_shape=((BATCH_SIZE, T, 1)))
reload_pretrained_disc.load_weights(D_save_path)
reload_pretrained_disc.summary()

## Seq Gan original Traning

In [ ]:
def train_generator(generator, discriminator, batch_size, T, mc_size=10, verbose=False,                   
                    optimizer=Adam(lr=0.001), end_token=0, init_token=1):
    
    prefix = tf.ones([batch_size, 1, 1], dtype=tf.float64)
    curr_state_et = prefix[:, -1:, :]
    
    with tf.GradientTape(persistent=True) as tape:                        

        gen_step_loss = []

        for _ in tqdm(range(1, T)):

            # MC rollout to the end to get final return
            rollout_returns = []
            for _ in range(mc_size):
                batch_rollout_et = rollout_from_prefix(generator, batch_size, prefix, T)
                rollout_returns.append(discriminator(batch_rollout_et))
            
            Q_avg = tf.reduce_mean(rollout_returns, axis=0)
            
            # one-step rollout to get next prefix
            token_logits = generator(prefix)        
            sampled_et = tf.random.categorical(token_logits, num_samples=1, dtype=tf.int32)
            sampled_et = tf.reshape(sampled_et, [batch_size, 1, 1])
            sampled_et = tf.cast(sampled_et, dtype=tf.float64)

            # stop genererating once hit end_token
            cond_end_token = tf.equal(curr_state_et, end_token)
            curr_state_et = tf.where(cond_end_token, curr_state_et, sampled_et)
            prefix = tf.concat([prefix, curr_state_et], axis=1)
            
            # averge loss over batch at each rollout step: -E[log_prob * Q]
            policy_gradient_loss = -tf.reduce_mean(tf.math.log(tf.nn.softmax(token_logits)) * Q_avg)
            gen_step_loss.append(policy_gradient_loss)
        
        avg_gen_loss = tf.reduce_mean(gen_step_loss)
        
        if verbose:
            print('generator loss:{}'.format(avg_gen_loss))
            print('-----------------------')

    # update generator
    generator_grads = tape.gradient(avg_gen_loss, generator.trainable_variables)
    optimizer.apply_gradients(zip(generator_grads, generator.trainable_variables))

    # explicitly drop tape because persistent=True
    del tape

    return avg_gen_loss


def train_discriminator(real_data_batch, generator, discriminator, batch_size, T, verbose=False,
                        optimizer=Adam(lr=0.001)):

    # data prep
    real_labels = tf.ones((batch_size, 1))        

    fake_labels = tf.zeros((batch_size, 1))    
    fake_data_batch = np.array(generate_sequences(1, generator, batch_size, T))
    
    total_data = tf.concat([fake_data_batch, real_data_batch], axis=0)
    total_labels = tf.concat([fake_labels, real_labels], axis=0)


    # train the discriminator
    with tf.GradientTape() as tape:                                                           
        # train discriminator
        pred_prob = discriminator(total_data)

        # cross-entropy loss
        discriminator_loss = tf.reduce_mean(
            tf.keras.losses.binary_crossentropy(total_labels, pred_prob, from_logits=False))        

        if verbose:
            print('total discriminator loss:{}'.format(discriminator_loss))
            print('-----------------------')

    grads = tape.gradient(discriminator_loss, discriminator.trainable_variables)
    optimizer.apply_gradients(zip(grads, discriminator.trainable_variables))

    return discriminator_loss

## Start training from pretrained G and D

In [ ]:
model_save_dir = '/home/lun/project-basileus/multitype-sequence-generation-by-tlstm-gan/models/seqgan_original'
G_save_path = model_save_dir + '/pretrained_gen_weights/model.tf'
D_save_path = model_save_dir + '/pretrained_disc_weights/model.tf'

G = build_G(
    batch_size = BATCH_SIZE,
    event_vocab_dim = EVENT_VOCAB_DIM,
    emb_dim = EMB_DIM,
    hidden_dim= HIDDEN_DIM,
)

G.build(input_shape=((BATCH_SIZE, T, 1)))
# G.load_weights(G_save_path)
G.summary()

D = build_D(T=T,
            event_vocab_dim=EVENT_VOCAB_DIM,
            emb_dim=EMB_DIM,
            num_filters=100,
            kernel_size=4,
            dropout_rate=0.25)

D.build(input_shape=((BATCH_SIZE, T, 1)))
# D.load_weights(D_save_path)
D.summary()

In [ ]:
N_total_G = pos_event_type_seqs.shape[0] // 20

EPOCHS = 3
_TOTAL_STEPS = int(EPOCHS * N_total_G / BATCH_SIZE)

train_dataset = create_dataset(pos_event_type_seqs[:N_total_G,:,:],
                             np.ones((N_total_G, 1)),
                             batch_size=BATCH_SIZE,
                             epochs=EPOCHS,
                             buffer_size=N_total_G)

gen_loss_history = []
disc_loss_history = []

step = 0

G_optimizer = SGD(learning_rate=1e-3)
D_optimizer = SGD(learning_rate=1e-3)

_G_STEPS = 1
_D_STEPS = 1

In [ ]:
for event_type_batch, _ in tqdm(train_dataset.take(_TOTAL_STEPS)):
    step += 1
    print('Training Step:', step)
    # train the generator
    for _ in range(_G_STEPS):
        gen_loss = train_generator(G, D, BATCH_SIZE, T, mc_size=100, verbose=True, optimizer=Adam(lr=0.001))        
        gen_loss_history.append(gen_loss.numpy())
    
    # train the discriminator
    for _ in range(_D_STEPS):
        disc_loss = train_discriminator(event_type_batch, G, D, BATCH_SIZE, T, verbose=True, optimizer=Adam(lr=0.001))
        disc_loss_history.append(disc_loss.numpy())

#     # save weights every 200 steps
#     if step % 100 == 0:
#         print('Saving weights...')
#         save_path_prefix = f'/home/lun/project-basileus/seq-gan/experiment_results/long_seqs_v10/oracle_train_{step}'
#         save_model_weights(save_path_prefix, generator, discriminator, critic)
#         print('All Saved!')

In [ ]:
len(gen_loss_history), len(disc_loss_history)

In [ ]:
x = range(len(gen_loss_history))
plt.figure(dpi=100)
plt.plot(x, gen_loss_history)
plt.title('Generator Loss History')
plt.xlabel('Training steps')

x = range(len(disc_loss_history))
plt.figure(dpi=100)
plt.plot(x, disc_loss_history)
plt.title('Discriminator Loss History')
plt.xlabel('Training steps')

In [ ]:
a_exam = generate_sequences(1000, G, BATCH_SIZE, T)

In [ ]:
G0 = build_G(
    batch_size = BATCH_SIZE,
    event_vocab_dim = EVENT_VOCAB_DIM,
    emb_dim = EMB_DIM,
    hidden_dim= HIDDEN_DIM,
)
b_exam = generate_sequences(1000, G0,BATCH_SIZE, T)

In [ ]:
model_save_dir = '/home/lun/project-basileus/multitype-sequence-generation-by-tlstm-gan/models/seqgan_original'
if not os.path.exists(model_save_dir + '/G_run_0'):
    os.makedirs(model_save_dir + '/G_run_0')

G_save_path = model_save_dir + '/G_run_0/model.tf'
                
if not os.path.exists(model_save_dir + '/D_run_0'):
    os.makedirs(model_save_dir + '/D_run_0')

D_save_path = model_save_dir + '/D_run_0/model.tf'

G.save_weights(G_save_path)
D.save_weights(D_save_path)

## Debug

In [ ]:
example_a = tf.zeros((64, 21, 1), dtype=tf.float32)

In [ ]:
rollout_returns = []
for _ in range(10):
    rollout_returns.append(D(example_a))

In [ ]:
tf.reduce_mean(rollout_returns, axis=0).shape

In [ ]:
logits = np.array([1.,2.,-1.]).reshape((1,3))
probs = tf.nn.softmax(logits)

In [ ]:
tf.random.categorical(logits,100)

In [ ]:
aa = tf.reshape(tf.math.log(probs), (1,3))
aa

In [ ]:
tf.random.categorical(aa,100)

### Debug the D of seq-gan

In [40]:
def build_D_1(T, event_vocab_dim, emb_dim, num_filters=100, kernel_size=4, dropout_rate=0.25):
    # Highway network
    i_et = Input(shape=(T, 1), name='event_type')  # input of discrete feature event type

    mask_layer = tf.keras.layers.Masking(mask_value=0., input_shape=(T, 1))
    masked_et = mask_layer(i_et)
    
    embed0 = Embedding(input_dim=event_vocab_dim, output_dim=emb_dim, mask_zero=True)(masked_et)
    embed0 = tf.squeeze(embed0, axis=2)

    conv1 = Conv1D(
        filters=num_filters,
        kernel_size=kernel_size,
        strides=1,
        padding="valid",
        activation='relu',
        use_bias=True,
        kernel_initializer=tf.keras.initializers.TruncatedNormal(mean=0.0, stddev=0.1, seed=None),
        bias_initializer=tf.keras.initializers.Constant(value=0.1))(embed0)
    
    pooled = MaxPooling1D(pool_size=T-kernel_size+1, strides=1)(conv1)
    
#     highwayed = highway_layers(pooled, n_layers=1, activation="relu", gate_bias=0)

#     dropped = Dropout(rate=dropout_rate)(highwayed)
    
    prob = Dense(1, 
                 activation='sigmoid',
                 name='final',
                 kernel_initializer=tf.keras.initializers.TruncatedNormal(mean=0.0, stddev=0.1, seed=None),
                 bias_initializer=tf.keras.initializers.Constant(value=0.1))(pooled)

    discriminator = Model(
        inputs=i_et,
        outputs=prob)

    return discriminator

In [34]:
def build_D_2(T, event_vocab_dim, emb_dim, num_filters=100, kernel_size=4, dropout_rate=0.25):    
    # normal LSTM
    i_et = Input(batch_shape=(BATCH_SIZE, T, 1), name='event_type')  # input of discrete feature event type
    mask_layer = tf.keras.layers.Masking(mask_value=0., input_shape=(None, 1))
    masked_et = mask_layer(i_et)
    
    embed0 = Embedding(input_dim=event_vocab_dim, output_dim=emb_dim, mask_zero=True)(masked_et)
    # turn [batch_size, None, 1, emb_dim] -> [batch_size, None, emb_dim]
    embed0 = tf.squeeze(embed0, axis=2)

    hm = LSTM(HIDDEN_DIM, 
              name='lstm',
              stateful=True,
              return_sequences=False,
              kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.1),
              recurrent_initializer=tf.keras.initializers.RandomNormal(stddev=0.1),
              bias_initializer=tf.keras.initializers.RandomNormal(stddev=0.1))(embed0)
              
    dropped = Dropout(rate=dropout_rate)(hm)
    
    prob = Dense(1, 
             activation='sigmoid',
             name='final',
             kernel_initializer=tf.keras.initializers.TruncatedNormal(mean=0.0, stddev=0.1, seed=None),
             bias_initializer=tf.keras.initializers.Constant(value=0.1))(dropped)  

    discriminator = Model(
        inputs=i_et,
        outputs=prob)

    return discriminator

In [35]:
pretrain_D_et = np.concatenate([neg_event_type_seqs[0:1000], pos_event_type_seqs[0:1000]], axis=0)
pretrain_D_labels = np.concatenate([np.zeros((1000, 1)), np.ones((1000, 1))], axis=0)

In [36]:
EPOCHS = 100
_TOTAL_STEPS = int(EPOCHS * 2000 / BATCH_SIZE)

pretrain_D_dataset = create_dataset(pretrain_D_et,
                                    pretrain_D_labels,
                                    batch_size=BATCH_SIZE,
                                    epochs=EPOCHS,
                                    buffer_size=2000) # shuffle the entire Dataset

pretrain_disc_token_loss_history = []

In [41]:
D1 = build_D_1(T=T,
             event_vocab_dim=EVENT_VOCAB_DIM,
             emb_dim=32,
             num_filters=1000,
             kernel_size=8,
             dropout_rate=0.1)

D1.summary()

Model: "model_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
event_type (InputLayer)      [(None, 21, 1)]           0         
_________________________________________________________________
masking_9 (Masking)          (None, 21, 1)             0         
_________________________________________________________________
embedding_9 (Embedding)      (None, 21, 1, 32)         192       
_________________________________________________________________
tf_op_layer_Squeeze_9 (Tenso [(None, 21, 32)]          0         
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 14, 1000)          257000    
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 1, 1000)           0         
_________________________________________________________________
final (Dense)                (None, 1, 1)              1001

In [42]:
D2 = build_D_2(T=T,
             event_vocab_dim=EVENT_VOCAB_DIM,
             emb_dim=32,
             num_filters=100,
             kernel_size=4,
             dropout_rate=0.1)

D2.summary()

Model: "model_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
event_type (InputLayer)      [(16, 21, 1)]             0         
_________________________________________________________________
masking_10 (Masking)         (16, 21, 1)               0         
_________________________________________________________________
embedding_10 (Embedding)     (16, 21, 1, 32)           192       
_________________________________________________________________
tf_op_layer_Squeeze_10 (Tens [(16, 21, 32)]            0         
_________________________________________________________________
lstm (LSTM)                  (16, 32)                  8320      
_________________________________________________________________
dropout_4 (Dropout)          (16, 32)                  0         
_________________________________________________________________
final (Dense)                (16, 1)                   33 

In [43]:
step = 0
OPTIMIZER = Adam(lr=1e-4)
for event_type_batch, labels in tqdm(pretrain_D_dataset.take(_TOTAL_STEPS)):
    step += 1
    print('Training Step:', step)

    disc_token_loss = pretrain_discriminator(event_type_batch, labels, D1, verbose=True, optimizer=OPTIMIZER)
    pretrain_disc_token_loss_history.append(disc_token_loss.numpy())

2it [00:00, 16.40it/s]

Training Step: 1
total discriminator loss:0.703651979079722
Training Step: 2
total discriminator loss:0.7161893712643832
Training Step: 3
total discriminator loss:0.7080960271285824
Training Step: 4
total discriminator loss:0.7043938026678822
Training Step: 5


8it [00:00, 21.73it/s]

total discriminator loss:0.7137545610206042
Training Step: 6
total discriminator loss:0.6934430025550327
Training Step: 7
total discriminator loss:0.6940960228014859
Training Step: 8
total discriminator loss:0.6914675280032275
Training Step: 9
total discriminator loss:0.6849716641258379


11it [00:00, 22.27it/s]

...

10275it [06:50, 23.02it/s]

Training Step: 10272
total discriminator loss:0.6925677713130585
Training Step: 10273
total discriminator loss:0.688840870502889
Training Step: 10274
total discriminator loss:0.6939721348086363
Training Step: 10275
total discriminator loss:0.6923667938500342
Training Step: 10276
total discriminator loss:0.6951559975179415


10276it [06:50, 25.04it/s]

Training Step: 10277
total discriminator loss:0.6910320107655741


KeyboardInterrupt: 